# <B> Create Git Repository using CodeCommit </B>
* Container: conda_python3

## AutoReload

In [1]:
%load_ext autoreload
%autoreload 2

## 1. parameter store 설정

In [9]:
import boto3
from utils.ssm import parameter_store

In [10]:
strRegionName=boto3.Session().region_name
pm = parameter_store(strRegionName)
strPrefix = pm.get_params(key="PREFIX")

In [11]:
strBucketName = pm.get_params(key="-".join([strPrefix, "BUCKET"]))
strExecutionRole = pm.get_params(key="-".join([strPrefix, "SAGEMAKER-ROLE-ARN"]))

In [12]:
print (f'strBucketName: {strBucketName}')
print (f'strExecutionRole: {strExecutionRole}')

strBucketName: sagemaker-us-east-1-419974056037
strExecutionRole: arn:aws:iam::419974056037:role/service-role/AmazonSageMaker-ExecutionRole-20221206T163436


## 2. CodeCommit 생성하기

In [17]:
import boto3

In [23]:
iam_client = boto3.client('iam')
codecommit_client = boto3.client('codecommit')

### 2.1 CodeCommit 관련 Credentials 생성 및 Parameter store에 저장하기

* CodeCommit Credentials

In [74]:
user_name = 'dongjin' ## ==> IAM에서 사용자 아이디 확인합니다.
codecommit_cred = 'codecommit-cred-'+user_name
print (f'codecommit_cred: {codecommit_cred}')

codecommit_cred: codecommit-cred-dongjin


In [75]:
try:
    response = iam_client.list_service_specific_credentials(
        UserName=user_name,
        ServiceName='codecommit.amazonaws.com'
    )
    if len(response['ServiceSpecificCredentials']) > 0:
        response = iam_client.delete_service_specific_credential(
            UserName=user_name,
            ServiceSpecificCredentialId=response['ServiceSpecificCredentials'][-1]['ServiceSpecificCredentialId']
        )
except:
    print("Create new codecommit crendentials")
    pass
finally:
    response = iam_client.create_service_specific_credential(
        UserName=user_name,
        ServiceName='codecommit.amazonaws.com'
    )
    ServiceUserName = response['ServiceSpecificCredential']['ServiceUserName']
    ServicePassword = response['ServiceSpecificCredential']['ServicePassword']
print(f"ServiceUserName : {ServiceUserName} \nServicePassword : {ServicePassword}")

ServiceUserName : dongjin-at-419974056037 
ServicePassword : 4ysElQd5UXHggw2mnXNnxVroQ5T99xzIN2QHjrOhxy0=


In [76]:
code_repository_name = 'sm-imd-pipeline-code' ## ==> 사용할 code repository 폴더 명을 넣습니다. ex) model_code
local_code_dir = './sources' ## ==> 생성한 local의 code repository 폴더 명을 넣습니다. ex) code

In [77]:
try:
    response = codecommit_client.create_repository(
        repositoryName=code_repository_name,
        repositoryDescription='Data Scientists share their training code using this Repository'
    )
except:
    
    print("Repository already exists")
    response = codecommit_client.get_repository(
        repositoryName=code_repository_name
    )

In [78]:
codecommit_repo = response['repositoryMetadata']['cloneUrlHttp']
codecommit_repo

'https://git-codecommit.us-east-1.amazonaws.com/v1/repos/sm-imd-pipeline-code'

In [79]:
!rm -rf .git/

!git init
!git remote add origin $codecommit_repo
!git checkout -b main
!git add $local_code_dir
!git commit -m "code-update"
!git push --set-upstream origin main

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /home/ec2-user/SageMaker/sagemaker-immersion-day-dl-framework/2.building-pipelines/.git/
Switched to a new branch 'main'
[main (root-commit) 9d3cee5] code-update
 11 files changed, 569 insertions(+)
 create mode 100644 sources/evaluation/.ipynb_checkpoints/requirements-checkpoint.txt
 create mode 100644 sources/evaluation/evaluation.py
 create mode 100644 sources/evaluation/requirements.txt
 create mode 100644 sources/preprocessing/.ipynb_checkpoints/preprocessing-checkpoint

## 3. parameter 저장하기

In [80]:
pm.put_params(key="-".join([strPrefix, "CODECOMMIT-USERNAME"]), value=ServiceUserName, overwrite=True, enc=True)
pm.put_params(key="-".join([strPrefix, "CODECOMMIT-PWD"]), value=ServicePassword, overwrite=True, enc=True)
pm.put_params(key="-".join([strPrefix, "CODE-REPO"]), value=codecommit_repo.replace('https://',''), overwrite=True)  ## https:// 있을 경우 입력 못함

'Store suceess'

In [81]:
print (f'CODE-REPO: {pm.get_params(key="-".join([strPrefix, "CODE-REPO"]), enc=False)}')
print (f'CODECOMMIT-USERNAME: {pm.get_params(key="-".join([strPrefix, "CODECOMMIT-USERNAME"]), enc=False)}')
print (f'CODECOMMIT-PWD: {pm.get_params(key="-".join([strPrefix, "CODECOMMIT-PWD"]), enc=False)}')

CODE-REPO: git-codecommit.us-east-1.amazonaws.com/v1/repos/sm-imd-pipeline-code
CODECOMMIT-USERNAME: AQICAHixC/mZVJcnQEHIgIK/d13m2pRN5MnNJb7dfKPh/9fZ0QFWLe/3h5oBlXYvV7gncefRAAAAdTBzBgkqhkiG9w0BBwagZjBkAgEAMF8GCSqGSIb3DQEHATAeBglghkgBZQMEAS4wEQQMzCggUD/QeRraTBLYAgEQgDLCFTCOvfiTvS78gkMr9r8m51o7iMVwfzNFTxrXdSVVS9Mf1eKL9NnhwF4DtWtMqm0F8g==
CODECOMMIT-PWD: AQICAHixC/mZVJcnQEHIgIK/d13m2pRN5MnNJb7dfKPh/9fZ0QEmWSJYaTKPjNmdqwTPBRShAAAAizCBiAYJKoZIhvcNAQcGoHsweQIBADB0BgkqhkiG9w0BBwEwHgYJYIZIAWUDBAEuMBEEDJju97XWF6YmjfsdEgIBEIBH6gp8xZuE9fY35ie9aJfY9s/dRduVKlUrBUoWxhzrtiS0jgPr7Z/0y2WF1AILfxuZ8+yaq5SR8Vfh155WXykJQG3khZfXze0=
